In [1]:
pip install googlesearch-python


[notice] A new release of pip is available: 24.0 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
from googlesearch import search
import re

def get_instagram_handles(name, max_results=10):
    query = f"{name} site:instagram.com"
    handles = set()

    for url in search(query, num_results=max_results):
        match = re.search(r"instagram\.com/([a-zA-Z0-9_.]+)/?", url)
        if match:
            handles.add(match.group(1))

    return list(handles)


In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
import time
from bs4 import BeautifulSoup
import re
import json

def scrape_bio_from_handle(handle):
    url = f"https://www.instagram.com/{handle}/"

    # Setup headless Chrome
    options = Options()
    options.add_argument("--headless")
    options.add_argument("--disable-gpu")
    options.add_argument("--window-size=1920,1080")
    driver = webdriver.Chrome(options=options)

    # Step 1: Open profile
    driver.get(url)
    time.sleep(5)  # Wait for the page to load

    # Step 2: Save page
    html = driver.page_source
    with open('test3.html', 'w', encoding='utf-8') as f:
        f.write(html)

    driver.quit()

    # Step 3: Parse with BeautifulSoup
    soup = BeautifulSoup(html, 'html.parser')

    # Extract meta tag with followers, following, posts
    desc_meta = soup.find('meta', attrs={'property': 'og:description'}) or soup.find('meta', attrs={'name': 'description'})
    desc_content = desc_meta['content'] if desc_meta else ''

    # Extract username and name from og:title
    title_meta = soup.find('meta', attrs={'property': 'og:title'})
    title_content = title_meta['content'] if title_meta else ''

    # Parse values using regex
    match = re.search(r"([\d.,A-Z]+) Followers, ([\d.,A-Z]+) Following, ([\d.,A-Z]+) Posts", desc_content)
    followers, following, posts = match.groups() if match else (None, None, None)

    username_match = re.search(r"\(@([^)]+)\)", title_content)
    username = username_match.group(1) if username_match else None

    name_match = re.match(r"^(.*?) \(@", title_content)
    name = name_match.group(1) if name_match else None

    profile_info = {
        "name": name,
        "username": username,
        "followers": followers,
        "following": following,
        "posts": posts,
        "profile_url": soup.find('meta', attrs={'property': 'og:url'})['content'] if soup.find('meta', attrs={'property': 'og:url'}) else url
    }

    return profile_info


In [ ]:
def match_profile_to_name(name, profiles):
    name = name.lower()
    for p in profiles:
        if p["name"] and name in p["name"].lower():
            return p
    return None


In [ ]:
def find_best_instagram_profile(name):
    handles = get_instagram_handles(name)
    profiles = []

    for h in handles:
        try:
            profile = scrape_bio_from_handle(h)
            if profile:
                profiles.append(profile)
        except Exception as e:
            print(f"Error scraping {h}: {e}")
            continue

    best_match = match_profile_to_name(name, profiles)
    return best_match if best_match else profiles  # return all if no perfect match


In [8]:
profile = find_best_instagram_profile("Ketan Arora")
print(profile)

[{'handle': 'ketanarora01', 'bio': '', 'full_name': ''}, {'handle': 'ketan_arora0611', 'bio': '', 'full_name': ''}, {'handle': 'k10arora', 'bio': '', 'full_name': ''}, {'handle': 'ketan.arora_', 'bio': '', 'full_name': ''}, {'handle': 'ketan_arora02', 'bio': '', 'full_name': ''}, {'handle': 'techbyketan', 'bio': '', 'full_name': ''}, {'handle': 'ketan.arora.90260', 'bio': '', 'full_name': ''}, {'handle': 'ketan_arora_', 'bio': '', 'full_name': ''}, {'handle': 'the_karan_arora_', 'bio': '', 'full_name': ''}, {'handle': 'aroraketan240', 'bio': '', 'full_name': ''}]


In [10]:
from googlesearch import search
import re
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
import time
from bs4 import BeautifulSoup
import json

def get_instagram_handles(name, max_results=10):
    query = f"{name} site:instagram.com"
    handles = set()

    for url in search(query, num_results=max_results):
        match = re.search(r"instagram\.com/([a-zA-Z0-9_.]+)/?", url)
        if match:
            handles.add(match.group(1))

    return list(handles)

def scrape_bio_from_handle(handle):
    url = f"https://www.instagram.com/{handle}/"

    # Setup headless Chrome
    options = Options()
    options.add_argument("--headless")
    options.add_argument("--disable-gpu")
    options.add_argument("--window-size=1920,1080")
    driver = webdriver.Chrome(options=options)

    # Step 1: Open profile
    driver.get(url)
    time.sleep(5)  # Let the page load fully

    # Step 2: Get HTML
    html = driver.page_source
    driver.quit()

    # Step 3: Parse with BeautifulSoup
    soup = BeautifulSoup(html, 'html.parser')

    # Extract meta tags
    desc_meta = soup.find('meta', attrs={'property': 'og:description'}) or soup.find('meta', attrs={'name': 'description'})
    desc_content = desc_meta['content'] if desc_meta else ''

    title_meta = soup.find('meta', attrs={'property': 'og:title'})
    title_content = title_meta['content'] if title_meta else ''

    # Regex parsing
    match = re.search(r"([\d.,A-Z]+) Followers, ([\d.,A-Z]+) Following, ([\d.,A-Z]+) Posts", desc_content)
    followers, following, posts = match.groups() if match else (None, None, None)

    username_match = re.search(r"\(@([^)]+)\)", title_content)
    username = username_match.group(1) if username_match else None

    name_match = re.match(r"^(.*?) \(@", title_content)
    name = name_match.group(1) if name_match else None

    profile_info = {
        "name": name,
        "username": username,
        "followers": followers,
        "following": following,
        "posts": posts,
        "profile_url": soup.find('meta', attrs={'property': 'og:url'})['content'] if soup.find('meta', attrs={'property': 'og:url'}) else url
    }

    return profile_info

# Run everything
def scrape_all_profiles_for_name(name):
    handles = get_instagram_handles(name)
    profiles = []

    for h in handles:
        try:
            print(f"Scraping: {h}")
            profile = scrape_bio_from_handle(h)
            profiles.append(profile)
        except Exception as e:
            print(f"Error scraping {h}: {e}")
    
    # Print nicely
    print(json.dumps(profiles, indent=2, ensure_ascii=False))
    return profiles


In [13]:
scrape_all_profiles_for_name("Aditya Kangude")

Scraping: aditya.kangude.77
Scraping: p
Scraping: adi.bagadi_
Scraping: aditya_______0039
Scraping: kangude_aditya10
Scraping: aditya_kangude_07
Scraping: aditya_kangude
Scraping: aditya.kangude.75
Scraping: kangude102
[
  {
    "name": "👑MR.ADITYA KANGUDE👑",
    "username": "aditya.kangude.77",
    "followers": "110",
    "following": "129",
    "posts": "0",
    "profile_url": "https://www.instagram.com/aditya.kangude.77/"
  },
  {
    "name": null,
    "username": null,
    "followers": null,
    "following": null,
    "posts": null,
    "profile_url": "https://www.instagram.com/p/"
  },
  {
    "name": "𝓐𝓭𝓲𝓽𝔂𝓪 𝓢𝓱𝓪𝓷𝓴𝓪𝓻 𝓫𝓪𝓰𝓪𝓭𝓲",
    "username": "adi.bagadi_",
    "followers": "48K",
    "following": "276",
    "posts": "485",
    "profile_url": "https://www.instagram.com/adi.bagadi_/"
  },
  {
    "name": "Aditya Kangude",
    "username": "aditya_______0039",
    "followers": "160",
    "following": "16",
    "posts": "45",
    "profile_url": "https://www.instagram.com/aditya_______0

[{'name': '👑MR.ADITYA KANGUDE👑',
  'username': 'aditya.kangude.77',
  'followers': '110',
  'following': '129',
  'posts': '0',
  'profile_url': 'https://www.instagram.com/aditya.kangude.77/'},
 {'name': None,
  'username': None,
  'followers': None,
  'following': None,
  'posts': None,
  'profile_url': 'https://www.instagram.com/p/'},
 {'name': '𝓐𝓭𝓲𝓽𝔂𝓪 𝓢𝓱𝓪𝓷𝓴𝓪𝓻 𝓫𝓪𝓰𝓪𝓭𝓲',
  'username': 'adi.bagadi_',
  'followers': '48K',
  'following': '276',
  'posts': '485',
  'profile_url': 'https://www.instagram.com/adi.bagadi_/'},
 {'name': 'Aditya Kangude',
  'username': 'aditya_______0039',
  'followers': '160',
  'following': '16',
  'posts': '45',
  'profile_url': 'https://www.instagram.com/aditya_______0039/'},
 {'name': 'Aditya Kangude',
  'username': 'kangude_aditya10',
  'followers': '1,001',
  'following': '1,207',
  'posts': '4',
  'profile_url': 'https://www.instagram.com/kangude_aditya10/'},
 {'name': 'Aditya Kangude',
  'username': 'aditya_kangude_07',
  'followers': '558',
  'followi